## Homework 4

Deployment


### Question 1

What's the version of pipenv you installed?

```
$ pipenv --version
> pipenv, version 2021.5.29
```

### Question 2

What's the first hash for scikit-learn you get in Pipfile.lock?

`sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829`


### Question 3

What's the probability that this customer is churning?

```
$ customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}
> churn probability 0.11549580587832914
```


### Question 4

What's the probability that this customer is churning?

```
$ customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
> {'churn': True, 'churn_probability': 0.9988892771007961}
```


### Question 5

When you build your image, what's the digest for `agrigorev/zoomcamp-model:3.8.12-slim?`

```
> bd897bb914af Pull complete
> [internal] load build definition from Dockerfile
```

### Question 6

What's the probability that this customer is churning?

```
$ customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
> {'churn': False, 'churn_probability': 0.32940789808151005}
```


## Scripts

>`predict.py`
```
import sys
import pickle

from flask import Flask
from flask import request
from flask import jsonify


model_path = 'model1.bin' #sys.argv[1]
dv_path = 'dv.bin' #sys.argv[2]

with open(model_path, 'rb') as model_in:
    model = pickle.load(model_in)

with open(dv_path, 'rb') as dv_in:
    dv = pickle.load(dv_in)

app = Flask('churn')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0,1]
    churn = y_pred > 0.5

    result = {
            'churn_probability': float(y_pred),
            'churn': bool(churn)
            }

    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)
```

>`Dockerfile`
```
FROM agrigorev/zoomcamp-model:3.8.12-slim

RUN pip install pipenv

WORKDIR /app
COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["predict.py", "model1.bin", "dv.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:app"]
```

>`Pipfile`
```
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
scikit-learn = "==1.0"
flask = "*"
gunicorn = "*"

[dev-packages]

[requires]
python_version = "3.8"
```
